In [530]:
from __future__ import division
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt

In [531]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [590]:
ratings_train = pd.read_csv('./homework4/data/ratings_train.csv', header=None)
ratings_test = pd.read_csv('./homework4/data/ratings_test.csv', header=None)
ratings_fake = pd.read_csv('./homework4/data/ratings_fake.csv', header=None)

In [591]:
ratings_train.columns = ['user', 'movie', 'rating']
ratings_test.columns = ['user', 'movie', 'rating']
ratings_fake.columns = ['user', 'movie', 'rating']

In [624]:
# A = ratings_fake.pivot(index='user', columns='movie').as_matrix()
A = ratings_train.pivot(index='user', columns='movie').as_matrix()

In [635]:
np.array(np.nan, 20)

TypeError: data type not understood

In [625]:
def optimize_U(U, V, B, C, mu, A, n, m):
    Uis = []
    for i in range(m):
        right = 0, 
        left = 0
        for j in range(n):
            right += (B[i] + C[j] + mu - A[i,j]) * V[j]
            left += np.outer(V[j], V[j].T)
        Ui = -np.dot(np.linalg.inv(left), right)
        Uis.append(Ui)
    return np.array(Uis)

In [626]:
def optimize_V(U, V, B, C, mu, A, n, m):
    Vjs = []
    for j in range(n):
        right = 0, 
        left = 0
        for i in range(m):
            right += (B[i] + C[j] + mu - A[i, j]) * U[i]
            left += np.outer(U[i], U[i].T)
        Vj = -np.dot(np.linalg.inv(left), right)
        Vjs.append(Vj)
    return np.array(Vjs)

In [627]:
def optimize_B(U, V, B, C, mu, A, n, m):
    Bis = []
    for i in range(m):
        b = 0
        for j in range(n):
            b += -(np.dot(U[i], V[j]) + C[j] + mu - A[i,j])
        Bis.append(b / n)
    return np.array(Bis)

In [628]:
def optimize_C(U, V, B, C, mu, A, n, m):
    Cjs = []
    for j in range(n):
        c = 0
        for i in range(m):
            c += -(np.dot(U[i], V[j]) + B[i] + mu - A[i,j])
        Cjs.append(c / m)
    return np.array(Cjs)

In [629]:
def update_params(B, U, C, V):
    B = optimize_B(U, V, B, C, mu, A)
    U = optimize_U(U, V, B, C, mu, A)
    C = optimize_C(U, V, B, C, mu, A)
    V = optimize_V(U, V, B, C, mu, A)
    return (B, U, C, V)

In [630]:
def log_likelihood(B, U, C, V, mu, A):
    lls = 0
    for i in range(U.shape[0]):
        for j in range(V.shape[0]):
            lls += (np.dot(U[i], V[j]) + B[i] + C[j] + mu - A[i,j])**2
    return -(lls.sum()) / 2

In [631]:
mu = A.mean()
m = A.shape[0]
n = A.shape[1]
k = 5
T = 20
B = np.zeros(m)
C = np.zeros(n)
U = np.random.multivariate_normal(np.zeros(k), (1/k) * np.identity(k), (m))
V = np.random.multivariate_normal(np.zeros(k), (1/k) * np.identity(k), (n))

In [632]:
v = []
for t in range(20):
    v.append(log_likelihood(B, U, C, V, mu, A))
    print v[t]
    B, U, C, V = update_params(B, U, C, V)
pd.Series(v).plot()

nan
nan


KeyboardInterrupt: 

In [607]:
def clip(z):
    if z < 1:
        return 1
    if z >= 1 and z <= 5:
        return z
    if z > 5:
        return 5